In [ ]:
import utility_module_2_v_3 as uu
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma
from numpy import meshgrid
from mpl_toolkits.basemap import Basemap

import geopandas as gpd
from shapely.geometry  import Point
from shapely.geometry import shape
import seaborn as sns
import pandas as pd 

import indices_no_leap as ii

## Import data

In [ ]:
gcm_name ='CanESM2'
output_figname = ''#_eqm'
path_out_IMD = '/home/vikram/BCSD_data/India_P_BCSDd/'

In [ ]:
gcm_meta=np.load(path_out_IMD + '/bcsd_imd_metadata_'+gcm_name+'.npy',allow_pickle=True)
Nyears_train = gcm_meta.item()['Nyears_train']


st=gcm_meta.item()['st']
en=gcm_meta.item()['en']

lat_obs=gcm_meta.item()['lat_obs']
lat_gcm_01=gcm_meta.item()['lat_gcm']
lon_obs=gcm_meta.item()['lon_obs']
lon_gcm_01=gcm_meta.item()['lon_gcm']
mask=gcm_meta.item()['mask']

imd_test=uu.correct_nan_inf_neg(np.load(path_out_IMD + '/data_obs_imd_pred.npy'))[365*Nyears_train:]

N1=imd_test.shape[1]
N2=imd_test.shape[2]





In [ ]:
from tqdm import tqdm
def check_for_nans_infs_zeros(data_sts):
    print('Nans:',np.sum(np.isnan(data_sts)))
    print('Infs:' ,np.sum(np.isinf(data_sts)))
    print('Zeros:',np.sum((data_sts==0)))
    
    
def spatial_ind(func,data_all,mask):
  N=data_all[0].shape[0]
  N1=data_all[0].shape[1]
  N2=data_all[0].shape[2]
  Nyears  =  int(N/365 )
  D=len(data_all)
  calc_index=np.zeros((D,N1,N2))
  for d in tqdm(range(D)):
        for i in range(N1):
          for j in range(N2):
            if mask[i,j] == 0:
                calc_index[d,i,j]=np.nanmean(func(data_all[d][:,i,j],Nyears))
  return calc_index


def spatial_ind_single(func,data_all,mask):
  N=data_all.shape[0]
  N1=data_all.shape[1]
  N2=data_all.shape[2]

  Nyears  =  int(N/365 )
    
  calc_index=np.zeros((N1,N2))
  for i in range(N1):
      for j in range(N2):
        if mask[i,j] == 0:
            calc_index[i,j]=np.nanmean(func(data_all[:,i,j],Nyears))
  return calc_index


In [ ]:
def S_score_2D(data_ref, data_model,mask_inf):
    # mask is true for exterior values
    data_1 = data_ref
    data_2 = data_model 
    ind_1 = np.isnan(data_1)
    ind_2 = np.isnan(data_2)
    data_1[ind_2] = np.nan
    data_2[ind_1] = np.nan
    data_1[mask_inf == True] = np.nan
    data_2[mask_inf == True] = np.nan

    data_1 = data_1.flatten()
    data_2 = data_2.flatten()
    
    data_1 =  np.delete(data_1,np.isnan(data_1))
    data_2 =  np.delete(data_2,np.isnan(data_2))
    R = np.corrcoef(data_1,data_2)[0,1]

    SDR = np.std(data_2)/np.std(data_1)

    S_score = (1+R)**4  / ( 4 * (SDR + 1 / SDR)**2 )
    
    
    return S_score

def RMSE_2D(data_ref, data_model,mask_inf):
    # mask is true for exterior values
    data_1 = data_ref
    data_2 = data_model 
    ind_1 = np.isnan(data_1)
    ind_2 = np.isnan(data_2)
    data_1[ind_2] = np.nan
    data_2[ind_1] = np.nan
    data_1[mask_inf == True] = np.nan
    data_2[mask_inf == True] = np.nan

    data_1 = data_1.flatten()
    data_2 = data_2.flatten()
    
    data_1 =  np.delete(data_1,np.isnan(data_1))
    data_2 =  np.delete(data_2,np.isnan(data_2))
    R = np.corrcoef(data_1,data_2)[0,1]
    return np.mean((data_2-data_1)**2)


In [ ]:
gcm_list = ['GFDL_CM3','CanESM2','MIROC_ESM','MRI_ESM1','ACCESS1_0','NorESM1_M','CM5A_LR','CNRM_CM5','MPI_ESM_MR']
index_list = ['Mean','CDD','Rx1day','R20mm','R99p']

In [ ]:
RMSE_bcsd = {}
RMSE_random = {}

S_score_bcsd = {}
S_score_random = {}
for iitr,iindex in enumerate([ii.mean_annualy,ii.CDD_annualy,ii.Rx1day_annualy,ii.R20mm_annualy,ii.R99p_annualy,]):
    print(iitr)
    obs_ind = spatial_ind_single(iindex,imd_test,mask)
    
    S_score_random[iitr] = []
    RMSE_random[iitr] = []
    for j in tqdm(range(100)):
        out_id='pre_rnd_new_'+str(j)  
        # print(out_id)
        bcsd_rnd_test=uu.correct_nan_inf_neg((np.load(path_out_IMD + '/BCSD_outputs_'+out_id+'.npy')))[365*Nyears_train:]
        rnd_ind = spatial_ind_single(iindex,bcsd_rnd_test,mask)
        S_score_random[iitr].append(S_score_2D(obs_ind,rnd_ind,mask))
        RMSE_random[iitr].append(RMSE_2D(obs_ind,rnd_ind,mask))
        
    S_score_bcsd[iitr] = []
    RMSE_bcsd[iitr] = []
    
    for gcm_name in gcm_list:
        out_id='pre_'+gcm_name 
        bcsd_gcm_test=uu.correct_nan_inf_neg(np.load(path_out_IMD + '/BCSD_outputs_'+out_id+'.npy'))[365*Nyears_train:]
        bcsd_ind = spatial_ind_single(iindex,bcsd_gcm_test,mask)
        S_score_bcsd[iitr].append(S_score_2D(obs_ind,bcsd_ind,mask))
        RMSE_bcsd[iitr].append(RMSE_2D(obs_ind,bcsd_ind,mask))


In [ ]:
plt.figure(figsize = (8,4))
ax =plt.gca()

delmx =0.25
N_indices = 5
N_esm = 9
plt.boxplot(list(S_score_random[x] for x in range(N_indices)),widths = 0.1)
markers=['o','^','s','p','P','*','h','+','x','X','D']


plt.plot( 0,0, 'D',markersize = 10,color ='w',linestyle = 'None',mfc='none',label ='Random Fields')

# colors=['r','g','b','y']
for gi in range(N_esm):
    plt.plot( np.arange(1+delmx,N_indices + 1+delmx,1), list(S_score_bcsd[x][gi] for x in range(N_indices)),
             markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
    plt.xticks(np.arange(1+delmx*0.5,N_indices + 1+delmx*0.5,1),index_list)

    

        
        
plt.ylim([0.25,0.951]);
plt.ylabel("Score")
plt.xlabel("Indices")
ax =plt.gca()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0, pos.width * 0.9, pos.height])
ax.legend(loc='center right', bbox_to_anchor=(1.25, 0.5))
plt.xlim([1.7,N_indices+1-0.3])
# plt.figure(figsize = (6,4))
# delmx =0
# N_indices = 5
# N_esm = 9
# plt.boxplot(list(np.sqrt(RMSE_random[x]) for x in range(1,N_indices)))
# markers=['o','^','s','p','P','*','h','+','x','X','D']

# # colors=['r','g','b','y']
# for gi in range(N_esm):
#     plt.plot( np.arange(1+delmx,N_indices + delmx,1), list(np.sqrt(RMSE_bcsd[x][gi]) for x in range(1,N_indices)),
#              markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
#     plt.xticks(np.arange(1+delmx,N_indices + delmx,1),index_list[1:])
# # plt.ylim([0,500]);
plt.tight_layout()
plt.savefig('./Figures/NFigure4/ex_all_gcm_random.jpg',dpi = 300)
plt.savefig('./Figures/NFigure4/ex_all_gcm_random.svg')
plt.savefig("/home/vikram/Dropbox/BCSD Work/03_Figures/Figure_new_auroop/Figure2/part_2.svg")



In [ ]:
def prints(*args):
    for arg in args:
        print(arg.shape)

In [ ]:
np.random.randn(100,5).shape

In [ ]:

def box_plot_new(x_input,y_input,ax):
    for x_data,y_data in zip(x_input,y_input):
    #     x_data = 1
    # y_data = 10 +np.random.randn(100)

        box_width = 0.5


        p0   = np.percentile(y_data,0)
        p25  = np.percentile(y_data,25)
        p50  = np.percentile(y_data,50)
        p75  = np.percentile(y_data,75)
        p100 = np.percentile(y_data,100)


        box1 = [ (x_data - box_width/2 , p0),
                (x_data + box_width/2 , p0),
                (x_data + box_width/2 , p100),
                (x_data - box_width/2 , p100),
                (x_data - box_width/2 , p0) ]


        box2 = [ (x_data - box_width/2 , p25),
                (x_data + box_width/2 , p25),
                (x_data + box_width/2 , p75),
                (x_data - box_width/2 , p75),
                (x_data - box_width/2 , p25) ]



        poly = Polygon(box1,alpha=0.5)
        ax.add_patch(poly)
        poly = Polygon(box2,alpha =1)
        ax.add_patch(poly)


    plt.ylim([np.min(y_input)-1,np.max(y_input)+1])
    plt.xlim([np.min(x_input)-1,np.max(x_input)+1])





In [ ]:
plt.figure()
ax =plt.gca()

x_input = range(5)
y_input = np.random.randn(5,100)

box_plot_new(x_input,y_input,ax)

In [ ]:

RMSE_bcsd = {}
RMSE_random = {}
for iitr,iindex in enumerate([ii.mean_annualy,ii.CDD_annualy,ii.Rx1day_annualy,ii.R20mm_annualy,ii.R99p_annualy,]):
    print(iitr)
    obs_ind = spatial_ind_single(iindex,imd_test,mask)
    RMSE_random[iitr] = []
    for j in tqdm(range(10)):
        out_id='pre_rnd_new_'+str(j)  
        # print(out_id)
        bcsd_rnd_test=uu.correct_nan_inf_neg((np.load(path_out_IMD + '/BCSD_outputs_'+out_id+'.npy')))[365*Nyears_train:]
        rnd_ind = spatial_ind_single(iindex,bcsd_rnd_test,mask)
        RMSE_random[iitr].append(RMSE_2D(obs_ind,rnd_ind,mask))
    RMSE_bcsd[iitr] = []
    for gcm_name in gcm_list:
        out_id='pre_'+gcm_name 
        bcsd_gcm_test=uu.correct_nan_inf_neg(np.load(path_out_IMD + '/BCSD_outputs_'+out_id+'.npy'))[365*Nyears_train:]
        bcsd_ind = spatial_ind_single(iindex,bcsd_gcm_test,mask)
        RMSE_bcsd[iitr].append(RMSE_2D(obs_ind,bcsd_ind,mask))
        

In [ ]:
plt.figure(figsize = (6,4))
delmx =0
N_indices = 5
N_esm = 9
plt.boxplot(list(RMSE_random[x] for x in range(N_indices)))
markers=['o','^','s','p','P','*','h','+','x','X','D']

# colors=['r','g','b','y']
for gi in range(N_esm):
    plt.plot( np.arange(1+delmx,N_indices + 1+delmx,1), list(RMSE_bcsd[x][gi] for x in range(N_indices)),
             markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
    plt.xticks(np.arange(1+delmx,N_indices + 1+delmx,1),index_list)
# plt.ylim([0,1]);
# plt.legend()





In [ ]:
prints(imd_train,imd_test,data_gf_train,data_gf_test,bcsd_gcm_train,bcsd_gcm_test,bcsd_rnd_train,bcsd_rnd_test)

## Checking imported data

In [ ]:

data_train = [imd_train,data_gf_train,bcsd_gcm_train,bcsd_rnd_train]
data_test = [imd_test,data_gf_test,bcsd_gcm_test,bcsd_rnd_test]


plt.figure(figsize=(10,10))
for i in range(4):
  plt.subplot(4,4,i+1)
  plt.pcolor(data_train[i][250,:,:])
  plt.colorbar()

plt.figure(figsize=(20,10))
for i in range(4):
  plt.subplot(4,4,i+1)
  plt.plot(data_train[i][:,7,7])
    
plt.figure(figsize=(10,10))
for i in range(4):
  plt.subplot(4,4,i+1)
  plt.pcolor(data_test[i][250,:,:])
  plt.colorbar()

plt.figure(figsize=(20,10))
for i in range(4):
  plt.subplot(4,4,i+1)
  plt.plot(data_test[i][:,7,7])



## Compute indices

In [ ]:
from matplotlib.patches import Polygon


In [ ]:
def draw_screen_poly( x, y,c):
    
    

In [ ]:
points_x = [0,0,1,1,0]
points_y = [0,1,1,0,0]
c='r'

fig, ax = plt.subplots(1,1)

xy = zip(points_x,points_y)

poly = Polygon( list(xy),facecolor = c, alpha = 0.5)

plt.gca().add_patch(poly)
list(xy)

In [ ]:
from matplotlib.patches import Polygon
import matplotlib.pyplot as plt

polygon1 = Polygon([(0,5), (1,1), (3,0),])

fig, ax = plt.subplots(1,1)

ax.add_patch(polygon1)

plt.ylim(0,6)
plt.xlim(0,6)